# Install requirements

Create a requirements.txt and install the requirements.

In [ ]:
%%bash
echo 'torch
torchvision
' > requirements.txt

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

# Edit your code


## Edit and save your code file `main.py`.

In [ ]:
%%writefile main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)


if __name__ == '__main__':
    main()

## Test your code

In [ ]:
%run main.py

# Submit job on PAI

## Input PAI cluster auth info

In [ ]:
import os
user = os.environ['PAI_USER_NAME']
pai_url = os.environ['PAI_URL']

print(user)
import getpass
password = getpass.getpass()

## Configure your job

In [ ]:
# Generate a job id
import string
import random
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
job_name = f'mnist-pytorch-{id_generator()}'

job = {
    'jobName': job_name,
    # code_dir: will automatically generate code_dir and upload code
    'image': 'docker.io/openpai/job-editor',
    'taskRoles': [{
        'name': 'main',
        'taskNumber': 1,
        'cpuNumber': 4,
        'memoryMB': 8192,
        'command': f'sh -c "cd {job_name} && pip install -r requirements.txt && python main.py"'
    }],
    # 'authFile': f'{os.environ["HDFS_FS_DEFAULT"]}/user/{user}/authFile'
}
print(job)

## Submit your job

In [ ]:
# upload code
target_path = f'/user/{user}/{job_name}/main.py'
!hdfscli upload --alias=dev main.py {target_path}
target_path = f'/user/{user}/{job_name}/requirements.txt'
!hdfscli upload --alias=dev requirements.txt {target_path}
code_dir = f'{os.environ["HDFS_FS_DEFAULT"]}/user/{user}/{job_name}/'
job['codeDir'] = code_dir

# display job config
print('Job:', job)

# Get auth token
import requests
data = {"username": user, "password": password, "expiration": 3600}
url = f'{pai_url}/rest-server/api/v1/authn/basic/login'
response = requests.post(
    url, headers={'Content-Type': 'application/json'}, json=data)
# print(response.status_code)
token = response.json()["token"]
print('Token:', token)

# Submit job
create_job_url = f'{pai_url}/rest-server/api/v1/user/{user}/jobs'
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

response = requests.post(create_job_url, headers=headers, json=job)
print('Submit job:', response.json())
print('Job link:', f'{pai_url}/view.html?username={user}&jobName={job_name}')